<a href="https://colab.research.google.com/github/osoleole/colab/blob/master/TF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation
[Manual on Colab](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)
### Tensorflow 2
```python
!pip install tensorflow-gpu==2.0.0-rc0
```

In [0]:
#!pip install --upgrade tfp-nightly
!pip install tensorflow-gpu==2.0.0-rc0

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import xgboost

#import tensorflow_probability as tfp
#from tensorflow_probability import distributions as tfd
#from tensorflow_probability import sts

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
tf.debugging.set_log_device_placement(True)

In [0]:
## Enviroment
## Run to get information
from tensorflow.python.client import device_lib
print("Tensorflow version: {}".format(tf.__version__))
print("GPU Available: {}".format(tf.test.is_gpu_available()))
print("GPU device: {}".format(tf.test.gpu_device_name()))
print("Used device: {}".format(device_lib.list_local_devices()))

## memory
!cat /proc/meminfo
## cpu
!cat /proc/cpuinfo

In [0]:
# One hot encoder for categorical data
def one_hot_enc(y):
    y = y.reshape(-1,1)
    ohe = OneHotEncoder(categories='auto')
    ohe.fit(y)
    return ohe.transform(y).toarray()

# Train data 1
# Two blobs of normally distributed data
n_samples = 1000
n_features = 32
random_state = 10
x1, y1 = datasets.make_blobs(n_samples=n_samples, n_features=n_features, centers=3, 
           cluster_std=2.0, center_box=(-20.0, 20.0), 
           shuffle=True, random_state=random_state)
# Encode categories to one hot vectors
y1 = one_hot_enc(y1)

train_to_test = 0.9
loc = int(np.ceil(n_samples*train_to_test))
x1_train = x1[:loc]
y1_train = y1[:loc]
x1_test = x1[loc:]
y1_test = y1[loc:]

In [5]:
y1

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [0]:
batch = 64

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x1_train,y1_train))
train_ds = train_ds.batch(batch)
test_ds = tf.data.Dataset.from_tensor_slices((x1_test,y1_test))
test_ds = test_ds.batch(batch)

In [0]:
# Callbacks
class Info(keras.callbacks.Callback):
  def on_batch_end(self, batch, logs={}):
    print("Batch ended")

info_cb = Info()

In [0]:
# Functional API
inputs = tf.keras.Input(shape=(32,), name='input_1')
x = layers.Dense(32, activation='relu', name='dense_1')(inputs)
#x = layers.Dense(64, activation='sigmoid', name='dense_2')(x)
x = layers.BatchNormalization(axis=-1)(x)
predictions = layers.Dense(3, activation='softmax',name='output_1')(x)
# Model
fmodel = tf.keras.Model(inputs=inputs, outputs=predictions)
# Compile
fmodel.compile(optimizer=tf.keras.optimizers.Adam(0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
print(fmodel.summary())
#fmodel.layers[1].get_weights()[0].shape
#keras.utils.plot_model(fmodel, 'fmodel_1.png', show_shapes=True)

In [0]:
fmodel.fit(train_ds, epochs=10, callbacks=[])

In [0]:
tf.config.experimental.list_logical_devices()


In [0]:
range_ds = tf.data.Dataset.range(100000)

In [0]:
batches = range_ds.batch(10, drop_remainder=True)

In [15]:
for batch in batches.take(5):
  print(batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]


In [20]:
a = tf.constant([1.,2.,3.])
a[-1].numpy()

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0


3.0

In [0]:
# Sequential API
model = tf.keras.Sequential([
layers.Dense(10, activation='sigmoid', input_shape=(32,)),
#layers.Dense(10, activation='sigmoid'),
layers.Dense(3, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [0]:
# Train and validate
model.fit(train_ds, epochs=10, validation_data=test_ds)

In [0]:
model.evaluate(test_ds)

In [0]:
x1_test[0]

In [0]:
## Input data
in_1 = x1_test[0].reshape(1,-1)
print("Prediction:", model.predict(in_1))
print("True: ", y1_test[0] )

In [0]:
x1, y2 = datasets.make_blobs(n_samples=n_samples, n_features=n_features, centers=3, 
           cluster_std=2.0, center_box=(-20.0, 20.0), 
           shuffle=True, random_state=random_state)

In [0]:
y2

In [17]:
keras.utils.to_categorical(y2, num_classes=3)

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [25]:
keras.utils.normalize([3.,4.,5.,4.], axis=-1, order=2)

array([[0.36927447, 0.49236596, 0.61545745, 0.49236596]])